# 🚀 RoofVision AI - Training on Cloud GPU

Use this notebook to train your model on **Google Colab** or **Kaggle** for free using a T4 GPU.

### Steps:
1.  **Run All Cells** from top to bottom.
2.  Wait for training to finish (approx 10-15 mins for 10 epochs).
3.  **Download** the `antigravity_model.pt` file at the end.
4.  Upload it to your laptop's `solar 3/antigravity/backend/models/` folder.

In [ ]:
# 1. Install Dependencies
!pip install torch torchvision opencv-python-headless matplotlib pyyaml tqdm
import torch
print(f"Using GPU: {torch.cuda.get_device_name(0)}")

### 2. Upload Your Dataset
1. Click the **Files** icon on the left sidebar.
2. Drag and drop your **`dataset_train`** and **`dataset_valid`** folders here.
   * Tip: Zip them first (`dataset.zip`), upload, and run `!unzip dataset.zip` to be faster.

In [ ]:
# (Optional) Unzip if you uploaded a zip file
# !unzip dataset.zip

In [ ]:
# 3. Define the Model & Training Script
# (We paste the code directly here to avoid cloning complex repos)

import os
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import Dataset, DataLoader
import cv2
import numpy as np
from tqdm import tqdm

def get_model_instance_segmentation(num_classes):
    # Load pre-trained Mask R-CNN
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
    
    # Replace box predictor
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    
    # Replace mask predictor
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    
    return model

class SolarDataset(Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        # Load all images
        self.imgs = list(sorted(os.listdir(os.path.join(root, "images"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images", self.imgs[idx])
        label_path = os.path.join(self.root, "labels", self.imgs[idx].replace(".jpg", ".txt").replace(".png", ".txt"))
        
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape
        
        # Parse Labels (YOLO Format -> Mask)
        boxes, masks_list, labels = [], [], []
        
        if os.path.exists(label_path):
            with open(label_path) as f:
                for line in f.readlines():
                    parts = list(map(float, line.strip().split()))
                    class_id = int(parts[0])
                    poly_points = parts[1:]
                    
                    pts = []
                    for i in range(0, len(poly_points), 2):
                        x = int(poly_points[i] * width)
                        y = int(poly_points[i+1] * height)
                        pts.append([x, y])
                    
                    if len(pts) < 3: continue
                    
                    # Create Mask
                    mask = np.zeros((height, width), dtype=np.uint8)
                    cv2.fillPoly(mask, [np.array(pts)], 1)
                    masks_list.append(mask)
                    
                    # Create Box
                    x_coords = [p[0] for p in pts]
                    y_coords = [p[1] for p in pts]
                    boxes.append([min(x_coords), min(y_coords), max(x_coords), max(y_coords)])
                    labels.append(1) # Class 1 = Solar Panel
        
        # Convert to Tensor
        target = {}
        if len(boxes) > 0:
            target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
            target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
            target["masks"] = torch.as_tensor(np.array(masks_list), dtype=torch.uint8)
        else:
             # Negative example (no solar)
            target["boxes"] = torch.zeros((0, 4), dtype=torch.float32)
            target["labels"] = torch.zeros((0,), dtype=torch.int64)
            target["masks"] = torch.zeros((0, height, width), dtype=torch.uint8)

        img = torchvision.transforms.functional.to_tensor(img)
        return img, target

    def __len__(self):
        return len(self.imgs)

def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# 4. Run Training
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Paths (Make sure these match where you uploaded/unzipped)
train_dir = "./dataset_train/" 
valid_dir = "./dataset_valid/"

dataset = SolarDataset(train_dir)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)

model = get_model_instance_segmentation(2)
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

num_epochs = 10 # <-- CHANGE EPOCHS HERE

print("Starting Training...")
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, targets in tqdm(data_loader):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        total_loss += losses.item()
        
    print(f"Epoch {epoch+1}/{num_epochs} finished. Avg Loss: {total_loss/len(data_loader)}")

# Save
torch.save(model.state_dict(), "antigravity_model.pt")
print("Model Saved! Run the next cell to download.")

In [ ]:
# 5. Download the Brain
from google.colab import files
files.download('antigravity_model.pt')